In [ ]:
!pip install vertexai

ERROR: Could not find a version that satisfies the requirement csv (from versions: none)
ERROR: No matching distribution found for csv


In [48]:
#this is for the unstructured data
import csv
import json, os
import time
from pathlib import Path
import vertexai
from vertexai.generative_models import GenerativeModel, Part
from google.cloud import storage
from google.cloud.storage import transfer_manager

project_id = "steady-service-448416-k2"
location = "us-central1"
bucket_name = "finance_economics_events"
raw_folder = " initial-loads/The unstructured data/raw"       # files which are downloaded by download_tsa_reports.py are written into this folder
split_folder = " initial-loads/The unstructured data/split/"   # input location for the extract function
llm_folder = " initial-loads/The unstructured data/llm_text/"  # output location for the extract function
model_name = "gemini-2.0-flash-exp"                # latest Gemini model as of 02/01/25
#prompt = "Convert the file to json format. Return the date, hour of day, airport code, airport name, city, state, checkpoint, and customer traffic."
prompt = """What are all the annual state energy consumption value listed in this TXT file? Extract the State name, Year, Value, and energy consumption Sector Name
Return the output as json with the schema State:string, Year:integer, Value:float, Sector:string.
Do not include any other fields."""

from google.cloud import storage

def split_documents(bucket_name, raw_folder, split_folder, max_lines=1000):
    storage_client = storage.Client()
    blobs = storage_client.list_blobs(bucket_name, prefix=raw_folder)

    for blob in blobs:
        if blob.name == raw_folder:
            continue

        source_filename = blob.name
        print("Downloading", source_filename)

        blob_data = blob.download_as_text()
        lines = blob_data.splitlines()

        start_line = 1
        file_count = 1
        buffer = []

        for line_num, line in enumerate(lines, 1):
            buffer.append(line)

            if len(buffer) >= max_lines:
                file_name = os.path.join(split_folder, f"{os.path.basename(blob.name).rsplit('.', 1)[0]}_{file_count}.txt")
                # file_path = f"{file_name}_{file_count}.txt"
                print("Writing", file_name)

                with open(file_name, "w", encoding="utf-8") as out:
                    out.write("\n".join(buffer))

                buffer = []
                file_count += 1
                start_line = line_num + 1

        print("Finished processing", source_filename)



def copy_to_GCS(local_folder, gcs_folder, file_extension):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)

    directory_as_path_obj = Path(local_folder)
    file_paths = directory_as_path_obj.rglob(f"{file_extension}")
    relative_paths = [path.relative_to(local_folder) for path in file_paths]
    string_paths = [str(path) for path in relative_paths]
    print("Found {} files.".format(string_paths))

    results = transfer_manager.upload_many_from_filenames(bucket, string_paths, source_directory=local_folder, blob_name_prefix=gcs_folder, max_workers=5)

    for name, result in zip(string_paths, results):

        if isinstance(result, Exception):
            print("Failed to upload {} due to exception: {}".format(name, result))
        else:
            print("Uploaded {} to {}.".format(name, bucket.name))


def main():

    vertexai.init(project=project_id, location=location)
    model = GenerativeModel(model_name)
    storage_client = storage.Client()

    gcs_in_folder = " initial-loads/The unstructured data/split"
    blobs = storage_client.list_blobs(bucket_name, prefix=gcs_in_folder)
    records = []

    for index, blob in enumerate(blobs):
        if not blob.name.endswith(".txt"):
            continue
        parts = blob.name.split("/")[-1].split("_")
        print(parts)
        name = parts[0]
        count = parts[-1].split(".")[0]


        os.makedirs(llm_folder, exist_ok=True)
        # output file
        csvfile = open(f"{llm_folder}/{name}_{count}.csv", "w", newline="\n")
        writer = csv.writer(csvfile, delimiter=",", quotechar="\"", quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["State","Year","Value","Sector"])

        text_file_uri = f"gs://{bucket_name}/{blob.name}"
        input_text = Part.from_uri(text_file_uri, "text/plain")
        resp = model.generate_content([input_text, prompt])
        resp_text = resp.text.replace("```json", "").replace("```", "").strip()

        # image_file = Part.from_uri(f"gs://{bucket_name}/{blob.name}", "application/pdf")
        # resp = model.generate_content([image_file, prompt])
        # resp_text = resp.text.replace("```json", "").replace("```", "").replace("\n", "")
        payload_list = resp_text.split("},")

        for payload in payload_list:
            print("payload:", payload)

            # # State
            # State_start = payload.find("State\":")
            # State_end = payload.find(",", State_start)
            # State = payload[State_start:State_end].replace("State", "").replace(":", "").replace("\"", "").strip()
            # #print("business:", business)

            # # Year
            # Year_start = payload.find("Year\":")
            # Year_end = payload.find(",", Year_start)
            # Year = payload[Year_start:Year_end].replace("Year", "").replace(":", "").replace("\"", "").strip()
            # #print("category:", category)

            # # Value
            # Value_start = payload.find("Value\":")
            # Value_end = payload.find("],", Value_start)
            # Value = payload[Value_start:Value_end].replace("Value", "").replace(":", "").replace("\"", "").replace("[", "").replace(",      ", ",").strip()

            # if Value == None:
            #     Value = ''
            # #print("menu_items:", menu_items)

            # # Sector
            # Sector_start = payload.find("Sector\":")
            # Sector_end = payload.find(",", Sector_start)
            # Sector = payload[Sector_start:Sector_end].replace("Sector", "").replace(":", "").replace("\"", "").replace("}", "").strip()
            #print("location:", location)
            # State
            State_start = payload.find('"State":') + len('"State":')
            State_end = payload.find(",", State_start)
            State = payload[State_start:State_end].replace(":", "").replace("\"", "").strip()

            # Year
            Year_start = payload.find('"Year":') + len('"Year":')
            Year_end = payload.find(",", Year_start)
            Year = payload[Year_start:Year_end].replace(":", "").replace("\"", "").strip()

            # Value
            Value_start = payload.find('"Value":') + len('"Value":')
            Value_end = payload.find(",", Value_start) if payload.find(",", Value_start) != -1 else payload.find("}", Value_start)
            Value = payload[Value_start:Value_end].replace(":", "").replace("\"", "").strip()

            if Value == "None":
                Value = ''

            # Sector
            Sector_start = payload.find('"Sector":') + len('"Sector":')
            Sector_end = payload.find("}", Sector_start) if payload.find(",", Sector_start) == -1 else payload.find(",", Sector_start)
            Sector = payload[Sector_start:Sector_end].replace(":", "").replace("\"", "").strip()
            # writer.writerow([airport_code, terminal, business, category, location, menu_items])
            print("check:", State,Year,Value,Sector)
            writer.writerow([State,Year,Value,Sector])

        csvfile.close()

if __name__ == "__main__":
    split_documents(bucket_name, raw_folder, split_folder) # split pdf documents due to large size
    copy_to_GCS(split_folder, split_folder, "*.txt") # copy split documents to GCS
    main()
    copy_to_GCS(llm_folder, llm_folder, "*.csv") # copy LLM output to GCS


Streaming output truncated to the last 5000 lines.
    "Sector": "electric power"
  
check: South Dakota 1989 132.0 electric power
payload: 
  {
    "State": "South Dakota",
    "Year": 1988,
    "Value": 223.0,
    "Sector": "electric power"
  
check: South Dakota 1988 223.0 electric power
payload: 
  {
    "State": "South Dakota",
    "Year": 1987,
    "Value": 77.0,
    "Sector": "electric power"
  
check: South Dakota 1987 77.0 electric power
payload: 
  {
    "State": "South Dakota",
    "Year": 1986,
    "Value": 41.0,
    "Sector": "electric power"
  
check: South Dakota 1986 41.0 electric power
payload: 
  {
    "State": "South Dakota",
    "Year": 1985,
    "Value": 26.0,
    "Sector": "electric power"
  
check: South Dakota 1985 26.0 electric power
payload: 
  {
    "State": "South Dakota",
    "Year": 1984,
    "Value": 52.0,
    "Sector": "electric power"
  
check: South Dakota 1984 52.0 electric power
payload: 
  {
    "State": "South Dakota",
    "Year": 1983,
    "Value"